In [1]:
import pickle, numpy
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10

Using TensorFlow backend.


## Load the data from Keras dataset<br>
Returns two tuples:<br>
X, X_test: uint8 array of RGB image data with shape (num_samples, 3, 32, 32).<br>
y, y_test: uint8 array of category labels (integers in range 0-9) with shape (num_samples,).

In [2]:
# the data, split between train and test sets
(X,y), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 16s 0us/step


In [3]:
# split training set into training and validation sets:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Write data to project filesystem

In [4]:
with open('cifar-10-tf-train.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('cifar-10-tf-valid.pkl', 'wb') as f:
    pickle.dump((X_valid, y_valid), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('cifar-10-tf-test.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
import ibm_boto3
from ibm_botocore.client import Config

## Insert Cloud Object Storage credentials 

1. Select the Data icon (upper right, icon with 1 and 0s)
2. Select Connections
3. Select the cell below and then select <b>Insert to code</b> for <b>Connection to project COS</b> to insert credentials
4. Rename the credentials (which are usually called credentials_1) to cos_credentials.   YOU WILL GET ERRORS IF YOU DO NOT DO THIS!

In [6]:

# @hidden_cell
cos_credentials = {
  'secret_key':'99a441d05931aebf4125a59dac23aaaa75972ad3c1562ce5',
  'iam_url':'https://iam.ng.bluemix.net/oidc/token',
  'api_key':'53hnW7r3j7GqbypbMoeErI2jejdkiRRSjgpjuNjRbLOJ',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:global:a/5b9e87012111056a33619f0b5dbb21d1:5745ac3f-6ddc-427f-9e6f-733419d40aa3::',
  'access_key':'3706f61388194b12b5550c9c3985b466',
  'url':'https://s3-api.us-geo.objectstorage.service.networklayer.com'
}


In [7]:
service_endpoint = cos_credentials['url']  

In [8]:
auth_endpoint = cos_credentials['iam_url']  

In [9]:
cos = ibm_boto3.client('s3',
                      ibm_api_key_id=cos_credentials['api_key'],
                      ibm_service_instance_id=cos_credentials['resource_instance_id'],
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

## Create bucket name -- I usually append my name to the end (i.e. cifar10-tutorials-joel)

In [10]:
bucket = "cifar10-tutorials-test"

## This call may fail if bucket has already been created -- that's fine, simply continue executing the next cells

In [11]:
cos.create_bucket(Bucket=bucket)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 16 Apr 2018 15:15:24 GMT',
   'server': 'Cleversafe/3.12.4.32',
   'x-amz-request-id': '44fe8213-a707-4298-9010-f12da8c4a271',
   'x-clv-request-id': '44fe8213-a707-4298-9010-f12da8c4a271',
   'x-clv-s3-version': '2.5'},
  'HTTPStatusCode': 200,
  'HostId': '',
  'RequestId': '44fe8213-a707-4298-9010-f12da8c4a271',
  'RetryAttempts': 0}}

In [12]:
files = ['cifar-10-tf-train.pkl',
         'cifar-10-tf-valid.pkl',
         'cifar-10-tf-test.pkl']

In [13]:
for file in files:
    print('Uploading data {}...', format(file))
    cos.upload_file(Filename=file, Bucket=bucket, Key=file)

Uploading data {}... cifar-10-tf-train.pkl
Uploading data {}... cifar-10-tf-valid.pkl
Uploading data {}... cifar-10-tf-test.pkl


In [14]:
cos.list_objects(Bucket=bucket)

{'Contents': [{'ETag': '"86e585d89b77f29ea770732cf8cd834a-4"',
   'Key': 'cifar-10-tf-test.pkl',
   'LastModified': datetime.datetime(2018, 4, 16, 15, 15, 35, 187000, tzinfo=tzlocal()),
   'Owner': {'DisplayName': '5745ac3f-6ddc-427f-9e6f-733419d40aa3',
    'ID': '5745ac3f-6ddc-427f-9e6f-733419d40aa3'},
   'Size': 30800251,
   'StorageClass': 'STANDARD'},
  {'ETag': '"73d9782cbf71a35257f6c05c3312b40e-15"',
   'Key': 'cifar-10-tf-train.pkl',
   'LastModified': datetime.datetime(2018, 4, 16, 15, 15, 30, 75000, tzinfo=tzlocal()),
   'Owner': {'DisplayName': '5745ac3f-6ddc-427f-9e6f-733419d40aa3',
    'ID': '5745ac3f-6ddc-427f-9e6f-733419d40aa3'},
   'Size': 122920213,
   'StorageClass': 'STANDARD'},
  {'ETag': '"073c708e9947de1be3b8dcd990ce2fcc-4"',
   'Key': 'cifar-10-tf-valid.pkl',
   'LastModified': datetime.datetime(2018, 4, 16, 15, 15, 32, 543000, tzinfo=tzlocal()),
   'Owner': {'DisplayName': '5745ac3f-6ddc-427f-9e6f-733419d40aa3',
    'ID': '5745ac3f-6ddc-427f-9e6f-733419d40aa3'},


## The number of the image we wish to test from the validation set

In [15]:
image_number = 110

In [16]:
with open("cifar-10-tf-valid.pkl", "rb") as f:
  cifar_test_data=pickle.load(f)
cifar_test_data[0][image_number].tolist()

[[[170, 196, 135],
  [189, 210, 156],
  [132, 150, 100],
  [128, 146, 95],
  [126, 145, 96],
  [70, 88, 48],
  [53, 71, 41],
  [46, 63, 40],
  [61, 77, 55],
  [57, 78, 53],
  [66, 93, 66],
  [78, 107, 80],
  [40, 59, 42],
  [48, 64, 46],
  [112, 130, 104],
  [106, 126, 92],
  [102, 125, 92],
  [104, 130, 99],
  [103, 129, 99],
  [95, 121, 92],
  [85, 110, 81],
  [83, 108, 80],
  [75, 95, 75],
  [27, 43, 32],
  [63, 76, 66],
  [141, 156, 125],
  [117, 131, 102],
  [62, 76, 52],
  [96, 111, 81],
  [104, 129, 101],
  [90, 123, 99],
  [86, 113, 93]],
 [[147, 172, 99],
  [146, 166, 99],
  [137, 152, 91],
  [177, 200, 130],
  [120, 148, 79],
  [77, 102, 51],
  [52, 72, 41],
  [54, 75, 52],
  [69, 96, 69],
  [60, 87, 60],
  [85, 113, 86],
  [87, 114, 87],
  [69, 89, 65],
  [40, 58, 33],
  [73, 93, 59],
  [92, 113, 73],
  [85, 108, 76],
  [76, 100, 75],
  [76, 100, 72],
  [67, 92, 62],
  [52, 75, 46],
  [49, 67, 43],
  [47, 61, 43],
  [25, 34, 25],
  [26, 33, 25],
  [66, 79, 58],
  [57, 71, 50

In [17]:
print (cifar_test_data[1][110])

[6]
